<h1>Run SEIRHUD</h1>

In [1]:
from model import SEIRHUD
import csv
import numpy as np
import pandas as pd
import time
import warnings
from tqdm import tqdm
import multiprocessing as mp
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("../data/data.csv")
data.head()

,dates,ts0,infec,leitos,uti,dth,dthcm
0,06/03/2020,8,1,0,0,0,0
1,07/03/2020,9,2,0,0,0,0
2,08/03/2020,10,2,0,0,0,0
3,09/03/2020,11,2,0,0,0,0
4,10/03/2020,12,2,0,0,0,0


In [3]:
def bootWeig(series, times):
    series = np.diff(series)
    series = np.insert(series, 0, 1)
    results = []
    for i in range(0,times):
        results.append(np.random.multinomial(n = sum(series), pvals = series/sum(series)))
    return np.array(results)

In [4]:
#using bootstrap to 
infeclists = bootWeig(data["infec"], 500)
deathslists = bootWeig(data["dthcm"], 500)

In [5]:
#Define empty lists to recive results
ypred = []
dpred = []
upred = []
hpred = []
spred = []
epred = []
beta1 = []
beta2 = []
gammaH = []
gammaU = []
delta = []
ia0 = []
t1 = []
e0 = []
is0 = []

In [6]:
#define fixed parameters:
kappa = 1/4
p = 0.2
gammaA = 1/3.5
gammaS = 1/4
muH = 0.15
muU = 0.4
xi = 0.53 
omega_U = 0.29 
omega_H = 0.14
N = 14873064
bound = ([0,0,0,1/14,1/14,0,0.05,0,0,0],
         [1.5,1,30,1/5,1/5,1,0.35,10/N,10/N,10/N])

In [8]:
for cases, deaths in tqdm(zip(infeclists, deathslists)):
    model = SEIRHUD( N, 1)
    model.fit(x = range(1,len(data["infec"]) + 1),
          y = np.cumsum(cases)+1,
          d = np.cumsum(deaths),
          bound = bound,
          kappa = kappa,
          p = p,
          gammaA = gammaA,
          gammaS = gammaS,
          muH = muH,
          muU = muU,
          xi = xi,
          omegaU = omega_U,
          omegaH = omega_H,
          stand_error = True,
          fittingByCumulativeCases=False,    
          )
    model.predict(200)
    
    coef = model.getCoef()
    
    #Append predictions
    ypred.append(model.ypred)
    dpred.append(model.D)
    spred.append(model.S)
    epred.append(model.E)
    
    
    #append parameters
    beta1.append(model.beta[0])
    beta2.append(model.beta[1])
    gammaH.append(model.gammaH)
    gammaU.append(model.gammaU)
    delta.append(model.delta)
    t1.append(model.dayBetaChange[0])
    ia0.append(model.ia0)
    e0.append(model.e0)
    is0.append(model.is0)


0it [00:00, ?it/s]

pyswarms.single.local_best:   0%|          |0/1000

pyswarms.single.local_best:   0%|          |1/1000

pyswarms.single.local_best:   0%|          |2/1000

pyswarms.single.local_best:   0%|          |3/1000

pyswarms.single.local_best:   0%|          |4/1000

pyswarms.single.local_best:   0%|          |5/1000

pyswarms.single.local_best:   1%|          |6/1000

pyswarms.single.local_best:   1%|          |7/1000

pyswarms.single.local_best:   1%|          |8/1000

pyswarms.single.local_best:   1%|          |9/1000

pyswarms.single.local_best:   1%|          |10/1000

pyswarms.single.local_best:   1%|          |11/1000

pyswarms.single.local_best:   1%|          |12/1000

pyswarms.single.local_best:   1%|▏         |13/1000

pyswarms.single.local_best:   1%|▏         |14/1000

pyswarms.single.local_best:   2%|▏         |15/1000

pyswarms.single.local_best:   2%|▏         |16/1000

pyswarms.single.local_best:   2%|▏         |17/1000

pyswarms.single.local_best:   2%|▏ 

posi=[1.17540368e+00 4.59035963e-01 1.93275040e+00 1.57922823e-01
 1.13842679e-01 5.96979387e-02 3.07842624e-01 5.36178929e-07
 6.60868133e-07 6.05767234e-07]
12.971502463647834



1it [57:24, 3444.40s/it]

pyswarms.single.local_best:   0%|          |0/1000

pyswarms.single.local_best:   0%|          |1/1000

pyswarms.single.local_best:   0%|          |2/1000

pyswarms.single.local_best:   0%|          |3/1000

pyswarms.single.local_best:   0%|          |4/1000

pyswarms.single.local_best:   0%|          |5/1000

pyswarms.single.local_best:   1%|          |6/1000

pyswarms.single.local_best:   1%|          |7/1000

pyswarms.single.local_best:   1%|          |8/1000

pyswarms.single.local_best:   1%|          |9/1000

pyswarms.single.local_best:   1%|          |10/1000

pyswarms.single.local_best:   1%|          |11/1000

pyswarms.single.local_best:   1%|          |12/1000

pyswarms.single.local_best:   1%|▏         |13/1000

pyswarms.single.local_best:   1%|▏         |14/1000

pyswarms.single.local_best:   2%|▏         |15/1000

pyswarms.single.local_best:   2%|▏         |16/1000

pyswarms.single.local_best:   2%|▏         |17/1000

pyswarms.single.local_best:  

posi=[6.38537995e-01 1.95633396e-01 2.03942730e+00 1.77678845e-01
 1.67758011e-01 4.21774384e-02 3.45733672e-01 4.29361212e-08
 6.00341778e-07 6.05395663e-07]
13.902969498027645



2it [1:45:53, 3283.90s/it]

pyswarms.single.local_best:   0%|          |0/1000

pyswarms.single.local_best:   0%|          |1/1000

pyswarms.single.local_best:   0%|          |2/1000

pyswarms.single.local_best:   0%|          |3/1000

pyswarms.single.local_best:   0%|          |4/1000

pyswarms.single.local_best:   0%|          |5/1000

pyswarms.single.local_best:   1%|          |6/1000

pyswarms.single.local_best:   1%|          |7/1000

pyswarms.single.local_best:   1%|          |8/1000

pyswarms.single.local_best:   1%|          |9/1000

pyswarms.single.local_best:   1%|          |10/1000

pyswarms.single.local_best:   1%|          |11/1000

pyswarms.single.local_best:   1%|          |12/1000

pyswarms.single.local_best:   1%|▏         |13/1000

pyswarms.single.local_best:   1%|▏         |14/1000

pyswarms.single.local_best:   2%|▏         |15/1000

pyswarms.single.local_best:   2%|▏         |16/1000

pyswarms.single.local_best:   2%|▏         |17/1000

pyswarms.single.local_best:

...

posi=[9.83199027e-01 3.91285131e-01 2.16308990e+00 1.65984469e-01
 1.73316441e-01 2.05559716e-02 3.01030033e-01 2.28428618e-07
 5.67029100e-07 5.96207841e-07]
15.311613730266568



9it [7:29:02, 2970.16s/it]

pyswarms.single.local_best:   0%|          |0/1000

pyswarms.single.local_best:   0%|          |1/1000

pyswarms.single.local_best:   0%|          |2/1000

pyswarms.single.local_best:   0%|          |3/1000

pyswarms.single.local_best:   0%|          |4/1000

pyswarms.single.local_best:   0%|          |5/1000

pyswarms.single.local_best:   1%|          |6/1000

pyswarms.single.local_best:   1%|          |7/1000

pyswarms.single.local_best:   1%|          |8/1000

pyswarms.single.local_best:   1%|          |9/1000

pyswarms.single.local_best:   1%|          |10/1000

pyswarms.single.local_best:   1%|          |11/1000

pyswarms.single.local_best:   1%|          |12/1000

pyswarms.single.local_best:   1%|▏         |13/1000

pyswarms.single.local_best:   1%|▏         |14/1000

pyswarms.single.local_best:   2%|▏         |15/1000

pyswarms.single.local_best:   2%|▏         |16/1000

pyswarms.single.local_best:   2%|▏         |17/1000

pyswarms.single.local_best:

posi=[5.97979923e-01 4.78719649e-01 1.70032691e+00 1.59850100e-01
 1.95332975e-01 1.13057928e-01 3.48554547e-01 3.41442783e-07
 5.85437531e-07 4.78856797e-07]
14.94137661356877



10it [8:18:08, 2962.83s/it]

pyswarms.single.local_best:   0%|          |0/1000

pyswarms.single.local_best:   0%|          |1/1000

pyswarms.single.local_best:   0%|          |2/1000

pyswarms.single.local_best:   0%|          |3/1000

pyswarms.single.local_best:   0%|          |4/1000

pyswarms.single.local_best:   0%|          |5/1000

pyswarms.single.local_best:   1%|          |6/1000

pyswarms.single.local_best:   1%|          |7/1000

pyswarms.single.local_best:   1%|          |8/1000

pyswarms.single.local_best:   1%|          |9/1000

pyswarms.single.local_best:   1%|          |10/1000

pyswarms.single.local_best:   1%|          |11/1000

pyswarms.single.local_best:   1%|          |12/1000

pyswarms.single.local_best:   1%|▏         |13/1000

pyswarms.single.local_best:   1%|▏         |14/1000

pyswarms.single.local_best:   2%|▏         |15/1000

pyswarms.single.local_best:   2%|▏         |16/1000

pyswarms.single.local_best:   2%|▏         |17/1000

pyswarms.single.local_best

KeyboardInterrupt: 

In [8]:
def getConfidenceInterval(series, length):
    series = np.array(series)
    
    #Compute mean value
    meanValue = [np.mean(series[:,i]) for i in range(0,length)]

    #Compute deltaStar
    deltaStar = meanValue - series

    #Compute lower and uper bound
    deltaL = [np.quantile(deltaStar[:,i], q = 0.025) for i in range(0,length)]
    deltaU = [np.quantile(deltaStar[:,i], q = 0.975) for i in range(0,length)]

    #Compute CI
    lowerBound  = np.array(meanValue) + np.array(deltaL)
    UpperBound  = np.array(meanValue) + np.array(deltaU)
    return [meanValue, lowerBound, UpperBound]

In [9]:
#Get confidence interval for prediction
for i, pred in tqdm(zip([ypred, dpred, upred, hpred, epred, spred],
                   ["Infec", "deaths", "UTI", "Hosp", "exposed", "susceptible"])):
    Meanvalue, lowerBound, UpperBound = getConfidenceInterval(i, len(data["infec"]) + 199)
    df = pd.DataFrame.from_dict({pred + "_mean": Meanvalue,
                            pred + "_lb": lowerBound,
                            pred + "_ub": UpperBound})
    df.to_csv("../results/" + pred + ".csv", index = False)

6it [00:00, 18.07it/s]


In [10]:
#Exprort parametes
parameters = pd.DataFrame.from_dict({"beta1": beta1,
                                     "beta2": beta2,
                                     "gammaH": gammaH,
                                     "gammaU": gammaU,
                                     "delta": delta,
                                     "ia0":ia0,
                                     "e0": e0,
                                     "t1": t1,
                                     "is0":is0})
parameters.to_csv("../results/Parameters.csv", index = False)

In [19]:
len(bound[0])

10